In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import re
from nltk.stem.snowball import SnowballStemmer

print("Done...")

Done...


### Process of Formatting Data:
1. Move all sentences into binary feature vector, bigram model

2. Move all tags into useable format for additive model accuracy

3. Run data through one of the following models:

    a. Recurrent Neural Network

    b. Heirarchical Clustering Method

    c. Decision Tree Regressor

### Part 1: Moving Sentences into Binary Feature Vector, UNI & BI models
#### Things made for training include...
    - descriptions_train

    - features_train (useless)

    - images_train

    - tags_train
#### Things made for testing include...
    - descriptions_test

    - features_trest

    - images_test

    - tags_test

### Loading in the Data

In [2]:
#Train_descriptions Path goes here
train_descriptions = './all_data/descriptions_train'
files = []
#Iterate over every file in the directory
for i in range(10000):
    #append said file to list
    filename = str(i)+'.txt'
    files.append(filename)
#Make sure we did not miss anything
print(len(files))
print(files[:50])
def load_data():
    d = []
    for f in files:
        sentences_per_file = []
        for line in open(train_descriptions+'/'+str(f)):
            #stripping punctuation from the line
            line = re.sub(r'[^\w\s]','',line)
            #splitting the line into a list based on spaces
            line = line.split(' ')
            #removing the last two characters of the last item in the list
            line[-1] = line[-1][:-2]
            #appending the cleaned line to our array
            sentences_per_file.append(line)
        d.append(sentences_per_file)
        #print(sentences_per_file)
    #appending each chunk of sentences per file to our total raw train data
    return d

raw_train_descriptions = load_data()
#printing length. Length should be equal to our number of files
print(len(raw_train_descriptions))

10000
['0.txt', '1.txt', '2.txt', '3.txt', '4.txt', '5.txt', '6.txt', '7.txt', '8.txt', '9.txt', '10.txt', '11.txt', '12.txt', '13.txt', '14.txt', '15.txt', '16.txt', '17.txt', '18.txt', '19.txt', '20.txt', '21.txt', '22.txt', '23.txt', '24.txt', '25.txt', '26.txt', '27.txt', '28.txt', '29.txt', '30.txt', '31.txt', '32.txt', '33.txt', '34.txt', '35.txt', '36.txt', '37.txt', '38.txt', '39.txt', '40.txt', '41.txt', '42.txt', '43.txt', '44.txt', '45.txt', '46.txt', '47.txt', '48.txt', '49.txt']
10000


### Cleaning Up the Data Sentences ... 

Currently, the data is in the format:

    raw_train_descriptions = [[[s1a], [s1b], [s1c], [s1d], [s1e]], ...[[sNa], [sNb], [sNc], [sNd], [sNe]]]]
    
We want to change the data from the above format to: 

    stemmed_train_per_image = [[s1a-e], ... [sNa-e]]
    
    Where every word in every sentence has been stemmed, stop words removed, and each block of 5 sentences concatenated into 1. 

In [3]:
#Initialized stemmer object and language
stemmer = SnowballStemmer('english')

#Running list of stop words, can add more...
stop_words = ['a', 'and', 'but', 'to', 'it', 
              'is', 'a', 'the', 'of', 'as', 
             'this', 'that', 'was']

def stem():
    '''
    Returns a cleaned, stemmed list of lists in the same format
    as the raw_train_descriptions. 
    Returns a list of all words seen in the dataset.
    
    Stemmer takes in a word like 'generously' and returns 'generous'
    from online sources, snowball seems to be a favorite!
    '''
    cleaned_words = []
    stemmed_train_descriptions = []
    for chunk in raw_train_descriptions:
        new_chunk = []
        for sentence in chunk:
            new_sentence = []
            for word in sentence:
                word = word.lower()
                if word not in stop_words:
                    clean = stemmer.stem(word)
                    cleaned_words.append(clean)
                    new_sentence.append(clean)
            new_chunk.append(new_sentence)
        stemmed_train_descriptions.append(new_chunk)
    return stemmed_train_descriptions, cleaned_words

stemmed_train, cleaned_words = stem()

print(len(stemmed_train), len(cleaned_words))

#Getting a set() of the cleaned_words for the feature vector
set_cleaned_words = list(set(cleaned_words))
print(len(set_cleaned_words))
print('LOL, only 8000 unique words!')

print('First 10 cleaned words...')
print(cleaned_words[:10])
print('\n\n') 
print('First 10 words in the set...')
print(set_cleaned_words[:10])

def condenser(list_of_lists):
    total = []
    for chunk in list_of_lists:
        chunk_of_words = []
        for sentence in chunk:
            for word in sentence:
                chunk_of_words.append(word)
        total.append(chunk_of_words)
    return total
stemmed_train_per_image = condenser(stemmed_train)
print('\nStemmed Train Words Per Each Image')
print(len(stemmed_train_per_image))
print(stemmed_train_per_image[:2])

(10000, 382838)
7997
LOL, only 8000 unique words!
First 10 cleaned words...
[u'skateboard', u'put', 'on', u'show', u'use', u'picnic', u'tabl', u'his', u'stag', u'skateboard']



First 10 words in the set...
['', u'fourfiv', u'birdfeed', u'childern', u'woodi', u'yellow', u'four', u'sunlit', u'digit', u'deli']

Stemmed Train Words Per Each Image
10000
[[u'skateboard', u'put', 'on', u'show', u'use', u'picnic', u'tabl', u'his', u'stag', u'skateboard', u'pull', u'trick', 'on', u'top', u'picnic', u'tabl', u'man', u'ride', 'on', u'skateboard', 'on', u'top', u'tabl', u'skate', u'boarder', u'do', u'trick', 'on', u'picnic', u'tabl', u'person', u'ride', u'skateboard', 'on', u'picnic', u'tabl', u'with', u'crowd', u'watchin'], [u'bowl', u'soup', u'has', u'some', u'carrot', u'shrimp', u'noodl', 'in', 'i', u'healthi', u'food', 'in', u'bowl', u'readi', u'eat', '', u'soup', u'has', u'carrot', u'shrimp', 'in', u'sit', u'next', u'chopstick', u'tasti', u'bowl', u'ramen', u'serv', u'for', u'someon', u'enjo

### Supporting Bigram Models

##### Okay, since we have everything set up, we need to make some easy adjustments to support the bigram modeling ...  We are taking our training set: 
    -[u'skateboard', u'put', 'on', u'show', u'use', u'picnic', u'tabl', u'his', u'stag', u'skateboard', u'pull', u'trick', 'on', u'top', u'picnic', u'tabl', u'man', u'ride', 'on', u'skateboard', 'on', u'top', u'tabl', u'skate', u'boarder', u'do', u'trick', 'on', u'picnic', u'tabl', u'person', u'ride', u'skateboard', 'on', u'picnic', u'tabl', u'with', u'crowd', u'watchin']
    
##### And we are transforming it into ... 
    -[u'skateboard put', u'put on', 'on show', ...]
    
    

In [4]:
bigram_setup = stemmed_train_per_image

#Getting pairs of words in DF
def bi_grammer(data, print_ans = False):
    '''
    Takes in: a list of lists
    List is a list of sentences corresponding to each image
    ie: List[0] describes 0.jpg and 0.txt in the training set
    
    Returns: a list of bigrammed words
            a list of bigrammed sentences
    '''
    #Initialize n_grams (list of bigrammed words)
    bi_grams = []
    #Initialize n_sentences (list of bigrammed sentences)
    bi_sentences = []
    
    #Iterating over all sentences in the dataset
    for sentence in data:
        #Initializing the temporary sentence holder
        bi_sentence = []
        for tup in enumerate(sentence):
            #Enumerate returns each word paired with its index
            index = tup[0]
            text = tup[1]
            try:
                #Try to add the word to the next one and append it 
                catch = tup[1]+' '+sentence[index+1]
                bi_grams.append(catch)
                bi_sentence.append(catch)
            except IndexError:
                #If you couldnt, that means you are at an edge!
                bi_grams.append(text)
                bi_sentence.append(text)
        #Append the bi_sentence to the total bi_sentences
        bi_sentences.append(bi_sentence)
    #Helper Print Statements if print_ans = True
    if print_ans:
        print('Total Amount of Bigram Words: %s\n'%len(bi_grams))
        print(bi_grams[:30])
        print('\nTotal Amount of Bigram Sentences: %s'%len(bi_sentences))
    return bi_grams, bi_sentences


bigram_words, bigram_train_descriptions = bi_grammer(bigram_setup, True)



Total Amount of Bigram Words: 382838

[u'skateboard put', u'put on', u'on show', u'show use', u'use picnic', u'picnic tabl', u'tabl his', u'his stag', u'stag skateboard', u'skateboard pull', u'pull trick', u'trick on', u'on top', u'top picnic', u'picnic tabl', u'tabl man', u'man ride', u'ride on', u'on skateboard', u'skateboard on', u'on top', u'top tabl', u'tabl skate', u'skate boarder', u'boarder do', u'do trick', u'trick on', u'on picnic', u'picnic tabl', u'tabl person']

Total Amount of Bigram Sentences: 10000


#### Making a set() of bigram words for the columns of the dataframe in the next step!!

In [8]:
set_bigram_words = set(bigram_words)
print('Bigram words reduced from len %s to %s' % (len(bigram_words), len(set_bigram_words)))
print('For contrast... the unigram set is only 8000 words long... oh boy...')

Bigram words reduced from len 382838 to 107520
For contrast... the unigram set is only 8000 words long... oh boy...


In [13]:
bigram = pd.DataFrame(0, index = range(len(raw_train_descriptions)), \
                      columns = set_bigram_words)
#runtime ~~ XX min

for col in bigram.columns:
    for row in range(len(bigram)):
        for biword in bigram_train_descriptions[row]:
            if col == biword:
                bigram[col][row] += 1

print(bigram.head())
bigram.to_csv('bigram')

      point u  while bicyclist  locat at  bed rough  member basebal  \
0  0        0                0         0          0               0   
1  0        0                0         0          0               0   
2  0        0                0         0          0               0   
3  0        0                0         0          0               0   
4  0        0                0         0          0               0   

   herd larg  yak giraff  veget lay  ski past     ...      chair eleg  \
0          0           0          0         0     ...               0   
1          0           0          0         0     ...               0   
2          0           0          0         0     ...               0   
3          0           0          0         0     ...               0   
4          0           0          0         0     ...               0   

   downward view  dessert sit  toss off  sit pizza  whit black  busi beac  \
0              0            0         0          0       

## Helper / Scratch Work (for checking and confirming logic only)

In [5]:
for i in range(len(stemmed_train_per_image[:5])):
    print '\n'
    print stemmed_train_per_image[i]
    print '\n'
    print raw_train_descriptions[i]



[u'skateboard', u'put', 'on', u'show', u'use', u'picnic', u'tabl', u'his', u'stag', u'skateboard', u'pull', u'trick', 'on', u'top', u'picnic', u'tabl', u'man', u'ride', 'on', u'skateboard', 'on', u'top', u'tabl', u'skate', u'boarder', u'do', u'trick', 'on', u'picnic', u'tabl', u'person', u'ride', u'skateboard', 'on', u'picnic', u'tabl', u'with', u'crowd', u'watchin']


[['The', 'skateboarder', 'is', 'putting', 'on', 'a', 'show', 'using', 'the', 'picnic', 'table', 'as', 'his', 'stag'], ['A', 'skateboarder', 'pulling', 'tricks', 'on', 'top', 'of', 'a', 'picnic', 'tabl'], ['A', 'man', 'riding', 'on', 'a', 'skateboard', 'on', 'top', 'of', 'a', 'tabl'], ['A', 'skate', 'boarder', 'doing', 'a', 'trick', 'on', 'a', 'picnic', 'tabl'], ['A', 'person', 'is', 'riding', 'a', 'skateboard', 'on', 'a', 'picnic', 'table', 'with', 'a', 'crowd', 'watchin']]


[u'bowl', u'soup', u'has', u'some', u'carrot', u'shrimp', u'noodl', 'in', 'i', u'healthi', u'food', 'in', u'bowl', u'readi', u'eat', '', u'soup',